In [38]:
class OTEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        super(OTEnv,self).__init__()
        self.newRecord = {'tradePosition':0.0,'deposit':0.0,
                          'pL':0.0,'roi':0.0,'runningPL':0.0,'runningRoi':0.0,'totalRealizedPL':0.0,'totalRealizedROI':0.0,
                         'daysOpen':0.0,'sequencesOpen':0.0} 

        self.idx_list = []
        self.totalroi = []
        self.totalpl = []
        self.sqz_open = []
        self.tradeCount = 0.0
        self.current_step = 0.0
        self.historicalWindow = 5
        self.totalRealizedPL = 0.0
        self.totalRealizedROI = 0.0
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # if 'cuda' in device.type:
        #     from google.colab import drive
        #     drive.mount('/content/drive',force_remount=False)
        #     self.data_directory = '/content/drive/Othercomputers/My MacBook Pro/DRLOT/Data.nosync/SPY/stableBaselines/'
        # else:
        #     self.data_directory = 'Data.nosync/SPY/stableBaselines/'
        # self.data_directory = '/content/drive/Othercomputers/My MacBook Pro/DRLOT/Data.nosync/SPY/stableBaselines/'
        self.data_directory = '/datasets/spy_calendar/'
        
        self.closed = False
        
        # if 'cuda' in device.type:
        self.minmax = MinMax3()
        # else:
            # self.minmax = MinMax2(self.data_directory,reset=False)

        self.min_deposit = self.minmax.minimum[self.minmax.columns.index('deposit_mark')]
        self.max_deposit = self.minmax.maximum[self.minmax.columns.index('deposit_mark')]
        self.min_tradePosition = 0.0
        self.max_tradePosition = 1.0
        self.min_pL = -25.0
        self.max_pL = 25.0
        self.min_Roi = -1.0
        self.max_Roi = 10.0
        self.min_sqz = 10.0
        self.max_sqz = 445.0
        self.min_days_open = self.minmax.minimum[self.minmax.columns.index('daysToExpiration_front')]
        self.max_days_open = self.minmax.maximum[self.minmax.columns.index('daysToExpiration_front')]

        self.files = sorted(glob.glob(self.data_directory+'*'))
        self.fileUsage = {x:1 for x in self.files}
        self.subFileUsage = {x:{} for x in self.files}
        self.width = pd.read_parquet(glob.glob(self.data_directory+'*')[0]).shape[-1] + len(self.newRecord)
        self.observation_space = Box(low=-2, high=2, shape=(self.historicalWindow*self.width,))
        self.action_space = Discrete(2)   

        self.seed()

        ## not sure if I should reset in the __init__
        self.reset()
        
    def _next_observation(self,):
        self.scaler_trade_record()
        
        ts_a = pd.DataFrame(self.trade_state_list_scaled).to_numpy(dtype=np.float32)
        self.obs = np.concatenate((self.minmax.scale(self.dflocked[0:self.historicalWindow]), ts_a), axis=1).flatten()
        
        # ts_t = torch.Tensor([[np.float32(v) for k,v in ts.items()] for ts in self.trade_state_list_scaled])
        # self.obs = torch.concat((self.minmax.scale(self.dflocked[self.current_step-self.historicalWindow:self.current_step]),ts_t),dim=1)
        
    def _take_action(self,):
        self.reward = 0.0
        if self.action == 1:
            if self.trade_record['tradePosition'] == 0:
                self.trade_record['tradePosition'] = 1
                self.tradeCount +=1
                self.db4exp = self.dflocked.iloc[self.current_step]['daysToExpiration_front']
                self.trade_record['deposit'] = round(random.uniform(self.dflocked.iloc[self.current_step]['bid_back'],self.dflocked.iloc[self.current_step]['ask_back']) - \
                                                     random.uniform(self.dflocked.iloc[self.current_step]['bid_front'],self.dflocked.iloc[self.current_step]['ask_front']),2)
                if self.trade_record['deposit'] < 0.01:
                    self.trade_record['deposit']=0.01

            elif self.trade_record['tradePosition'] == 1:
                self.trade_record['sequencesOpen'] += 1
                self.trade_record['daysOpen'] = self.db4exp - self.dflocked.iloc[self.current_step]['daysToExpiration_front']
                self.trade_record['pL'] = np.float32(self.dflocked.iloc[self.current_step]['deposit_mark'] - self.trade_record['deposit'])
                self.trade_record['roi'] = np.float32(self.trade_record['pL']/self.trade_record['deposit'])
                self.trade_record['runningPL'] = np.float32(self.trade_record['pL'] + self.trade_record['totalRealizedPL'])
                self.trade_record['runningRoi'] = np.float32(self.trade_record['roi'] + self.trade_record['totalRealizedROI'])
                
                self.reward += np.float32(self.trade_record['pL'])
                if self.done[self.current_step] == True:
                    self.reward -= 1000
        
        elif self.action == 0:
            if self.trade_record['tradePosition'] == 1:
                self.sqz_open.append(self.trade_record['sequencesOpen'])
                self.trade_record['tradePosition'] = 0
                self.trade_record['pL'] = round(random.uniform(self.dflocked.iloc[self.current_step]['bid_back'],self.dflocked.iloc[self.current_step]['ask_back']) - \
                                                random.uniform(self.dflocked.iloc[self.current_step]['bid_front'],self.dflocked.iloc[self.current_step]['ask_front']),2) - self.trade_record['deposit']                
                
                self.trade_record['roi'] = self.trade_record['pL']/self.trade_record['deposit']    
                self.trade_record['totalRealizedPL'] +=  self.trade_record['pL']                 
                self.trade_record['totalRealizedROI'] +=  self.trade_record['roi']                 
                self.trade_record['deposit'] = 0

                self.db4exp = 0
                self.closed = True

            elif self.trade_record['tradePosition'] == 0:
                pass  
            
        self.reward += np.float32(self.trade_record['totalRealizedPL'])
            
    def scaler_trade_record(self,):
        self.trade_state_list_scaled =[{key: value for key, value in x.items()} for x in self.trade_state_list] #self.trade_state_list.copy()
        for d in self.trade_state_list_scaled:
            d['tradePosition'] = (((d['tradePosition'] - self.min_tradePosition) / (self.max_tradePosition - self.min_tradePosition))*2)-1   
            d['deposit'] = (((d['deposit'] - self.min_deposit) / (self.max_deposit - self.min_deposit))*2)-1
            d['pL'] = (((d['pL'] - self.min_pL) / (self.max_pL - self.min_pL))*2)-1
            d['roi'] = (((d['roi'] - self.min_Roi ) / (self.max_Roi - self.min_Roi ))*2)-1
            d['runningPL'] = (((d['runningPL'] - self.min_pL) / (self.max_pL - self.min_pL))*2)-1    
            d['runningRoi'] = (((d['runningRoi'] - self.min_Roi ) / (self.max_Roi - self.min_Roi ))*2)-1    
            d['daysOpen'] = (((d['daysOpen'] - self.min_days_open) / (self.max_days_open - self.min_days_open))*2)-1    
            d['sequencesOpen'] = (((d['sequencesOpen'] - self.min_sqz ) / (self.max_sqz  - self.min_sqz ))*2)-1    

    def step(self, action):    
        self.current_step +=1
        self.action = action
        
        self._take_action()
                
        self.trade_state_list.append({key: value for key, value in self.trade_record.items()})
        self._next_observation()
        
        # display(pd.DataFrame(self.dflocked.iloc[self.current_step]).T[['WD','HOUR','daysToExpiration_front','bid_front','mark_front','ask_front','bid_back','mark_back','ask_back','deposit_mark','openInterest_front','openInterest_back','delta_front','delta_back']])
        # display(pd.DataFrame(self.trade_state_list))
        
        if self.closed == True:
            self.trade_record['pL'] = 0.0
            self.trade_record['roi'] = 0.0
            self.trade_record['daysOpen'] = 0.0
            self.trade_record['sequencesOpen'] = 0.0
            self.closed = False
        
        if self.done[self.current_step] == True:
            self.trade_record['runningPL'] = 0.0
            self.trade_record['runningRoi'] = 0.0 
            self.trade_record['totalRealizedPL'] = 0.0
            self.trade_record['totalRealizedROI'] = 0.0
            
        return self.obs, round(self.reward,4) , self.done[self.current_step], {}

    def reset(self,seed=1234,return_info=False): #options=**kwargs
        self.reward = 0.0
        self.trade_record = {key: value for key, value in self.newRecord.items()} 
        self.db4exp = 0
        self.trade_state_list = deque(maxlen=5)
        [self.trade_state_list.append({key: value for key, value in self.newRecord.items()} ) for _ in range(5)]
        self.current_step = self.historicalWindow - 1
        
        if len(self.idx_list) == 0:
            p_ = sum([v for k,v in self.fileUsage.items()])
            self.file = np.random.choice(list(self.fileUsage.keys()),p=[(v/p_) for k,v in self.fileUsage.items()])
            for k,v in self.fileUsage.items():
                if k != self.file:
                    self.fileUsage[k]+=1
            self.df = pd.read_parquet(self.file)
            self.idx_list = list(self.df.index.unique())

        self.idx = np.random.choice(self.idx_list)
        self.dflocked = self.df.loc[self.idx].sort_values(['daysToExpiration_front','HOUR'],ascending=[False,True])  
        self.idx_list.remove(self.idx)
        
        self.scaler_trade_record()
        
        ts_a = pd.DataFrame(self.trade_state_list_scaled).to_numpy(dtype=np.float32)
        self.obs = np.concatenate((self.minmax.scale(self.dflocked[0:self.historicalWindow]), ts_a), axis=1).flatten()
        
        # ts_t = torch.Tensor([[np.float32(v) for k,v in ts.items()] for ts in self.trade_state_list_scaled])
        # self.obs = torch.concat((self.minmax.scale(self.dflocked[0:self.historicalWindow]),ts_t),dim=1)
        
        self.done = [False for _ in range(self.dflocked.shape[0]-1)]+[True]
        self.max_steps = len(self.dflocked)
        return self.obs
        
    def render(self, mode='human', close=False):
        if len(self.totalpl)==0: tplmean = np.mean(self.totalpl)
        else:tplmean = 0
            
        if len(self.totalpl)==0: troimean = np.mean(self.totalroi)   
        else:troimean = 0
            
        print(f'Average PL: {round(tplmean,2)}, Average ROI: {round(troimean,2)}, Total Trades {self.tradeCount}')
        
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def close (self):
        pass


In [1]:
!git clone https://github.com/jeroencvlier/utils.git
!pip install -U -i https://test.pypi.org/simple/ gym-OT
import gym_OT

fatal: destination path 'utils' already exists and is not an empty directory.
Looking in indexes: https://test.pypi.org/simple/


In [2]:
import gym
from tqdm import tqdm

In [31]:
# env = gym.make("OT-v0")

from ray.tune.registry import register_env
import gym_OT
# select_env = "OT-v0"
# register_env(select_env, lambda config: OTGym())

In [4]:
import ray
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8,num_gpus=1)

2022-11-17 20:29:24,174	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.1.0


In [42]:
import gym, ray
from ray.rllib.algorithms import ppo

algo = ppo.PPO(env=OTEnv)

while True:
    print(algo.train())


ImportError: TensorFlow was specified as the 'framework' inside of your config dictionary. However, there was no installation found. You can install TensorFlow via `pip install tensorflow`

In [48]:
from ray.rllib.algorithms.dqn import DQN


In [68]:
DQN.get_default_config()['framework']="torch"

In [69]:
DQN

'tf'

In [67]:
DQN(config={"framework":"torch"})

ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!

In [62]:
from ray.tune.registry import register_env


register_env("my_env", OTEnv)
algo = DQN(env="my_env",config={"framework":"torch"})

TypeError: __init__() takes 1 positional argument but 2 were given

In [35]:
register_env("OT-v0", lambda config: gym_OT("OT-v0"))


In [36]:
import gym, ray
from ray.rllib.algorithms import ppo


register_env("my_env", gym_OT)


TypeError: ('Second argument must be callable.', <module 'gym_OT' from '/usr/local/lib/python3.8/dist-packages/gym_OT/__init__.py'>)

In [44]:
!rllib train --run DQN --env OT-v0 --framework torch

2022-11-17 20:48:49,880	WARNING utils.py:604 -- Ray currently does not support initializing Raywith fractional cpus. Your num_cpus will be truncated from 5.985 to 5.
2022-11-17 20:48:50,085	INFO worker.py:1528 -- Started a local Ray instance.
== Status ==
Current time: 2022-11-17 20:48:56 (running for 00:00:04.79)
Memory usage on this node: 4.0/29.4 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/5 CPUs, 0/1 GPUs, 0.0/14.89 GiB heap, 0.0/7.45 GiB objects (0.0/1.0 accelerator_type:P5000)
Result logdir: /root/ray_results/default
Number of trials: 1/1 (1 RUNNING)


2022-11-17 20:48:56,352	ERROR trial_runner.py:993 -- Trial DQN_OT-v0_3e06f_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/usr/local/lib/python3.8/dist-packages/ray/_private

In [7]:


from ray.rllib.algorithms.dqn import DQN

# Configure the algorithm.
config = {
    "env": "Taxi-v3",
    "num_workers": 7,
    "framework": "torch",
    "num_gpus" : 1,
    "model": {
        "fcnet_hiddens": [640, 640,300],
        "fcnet_activation": "relu",
    },
    "evaluation_num_workers": 1,
    "evaluation_interval":1,
    "evaluation_config": {
        "render_env": False,
    },
}

algo = DQN(config=config)

# for it in tqdm(range(30)):
#     algo.train()
#     print(it,'  ',algo.evaluate()['evaluation']['episode_reward_mean'])


# algo.evaluate()


(RolloutWorker pid=14127) 2022-11-17 18:39:32,920	WARNING env.py:159 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2022-11-17 18:39:33,559	WARNING deprecation.py:47 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=14319) 2022-11-17 18:39:37,491	WARNING env.py:159 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
2022-11-17 18:39:37,592	INFO trainable.py:164 -- Trainable.setup took 10.994 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [17]:
tune.TuneConfig

ray.tune.tune_config.TuneConfig

In [18]:
!pip3 install -U 'mujoco-py<2.2,>=2.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 kB 38.5 MB/s eta 0:00:00


In [24]:
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8,num_gpus=1)

import random

from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining

if __name__ == "__main__":

    # Postprocess the perturbed config to ensure it's still valid
    def explore(config):
        # ensure we collect enough timesteps to do sgd
        if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
            config["train_batch_size"] = config["sgd_minibatch_size"] * 2
        # ensure we run at least one sgd iter
        if config["num_sgd_iter"] < 1:
            config["num_sgd_iter"] = 1
        return config

    pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        
        # Specifies the mutations of these hyperparams
        hyperparam_mutations={
            "lambda": lambda: random.uniform(0.9, 1.0),
            # "clip_param": lambda: random.uniform(0.01, 0.5),
            "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "num_sgd_iter": lambda: random.randint(1, 30),
            "sgd_minibatch_size": lambda: random.randint(128, 16384),
            "train_batch_size": lambda: random.randint(2000, 160000),
        },
        custom_explore_fn=explore,
    )
    
    tuner = tune.Tuner(
        "DQN",
        tune_config=tune.TuneConfig(
            metric="episode_reward_mean",
            mode="max",
            scheduler=pbt,
            num_samples=1,
            reuse_actors=True,
            
            
        ),
        
        param_space={
            "env": "Taxi-v3",
            "framework": "torch",
            "num_workers": 7,
            "num_gpus": 1, # number of GPUs to use
            "model": {"free_log_std": True,
                      "fcnet_hiddens": [64, 64],
                      "fcnet_activation": "relu",
                     },
            # These params are tuned from a fixed starting value.
            # "lambda": 0.95,
            "clip_param": 0.2,
            "lr": 1e-4,
            # These params start off randomly drawn from a set.
            "num_sgd_iter": tune.choice([10, 20, 30]),
            "sgd_minibatch_size": tune.choice([128, 512, 2048]),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
            
        },
        
    )
    results = tuner.fit()

    print("best hyperparameters: ", results.get_best_result().config)

2022-11-17 17:25:21,548	INFO worker.py:1528 -- Started a local Ray instance.


2022-11-17 17:25:28,896	ERROR trial_runner.py:993 -- Trial DQN_Taxi-v3_d1bc5_00000: Error processing event.
ray.tune.error._TuneNoNextExecutorEventError: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/usr/local/lib/python3.8/dist-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=7783, ip=10.42.32.217, repr=DQN)
  File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable

Trial name,trial_id
DQN_Taxi-v3_d1bc5_00000,d1bc5_00000


(DQN pid=7783) 2022-11-17 17:25:28,873	ERROR worker.py:763 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=7783, ip=10.42.32.217, repr=DQN)
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
(DQN pid=7783)     super().__init__(config=config, logger_creator=logger_creator, **kwargs)
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/tune/trainable/trainable.py", line 161, in __init__
(DQN pid=7783)     self.setup(copy.deepcopy(self.config))
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 439, in setup
(DQN pid=7783)     self.config = self.merge_trainer_configs(
(DQN pid=7783)   File "/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py", line 2228, in merge_trainer_configs
(DQN pid=7783)     return deep_update(
(DQN pid=7783) Exception: Unknown config p

best hyperparameters:  None


In [7]:
a = algo.evaluate()


-532.1